In [1]:
library(FlashRLearn)
fm.set.conf("/FlashX/EC2/run_matrix.txt")
fm.set.test.na(FALSE)

Loading required package: pcg
Loading required package: FlashR
Loading required package: RSpectra
Loading required package: Rcpp

Attaching package: 'FlashR'

The following objects are masked from 'package:base':

    cbind, pmax, pmin, rbind



NULL

# Read a real data
Read from the very original source.

In [ ]:
data <- fm.read.obj("/mnt/data//Criteo_2days.mat")

Read from SSDs

In [2]:
emdata <- fm.get.dense.matrix("Criteo.mat")
data <- fm.conv.store(emdata, in.mem=TRUE)

In [3]:
y <- data[,1]
x <- data[,2:40]

In [ ]:
emdata <- fm.conv.store(data, in.mem = FALSE, name="Criteo.mat")

# Generate a synthetic data of similar size

In [ ]:
data <- fm.runif.matrix(325000000, 40, min = 0, max=1000)
data <- as.integer(data)
data <- fm.materialize(data)
emdata <- fm.conv.store(data, in.mem = FALSE, name="random.mat")

In [3]:
y <- fm.materialize(data[,1] > 0.5)
x <- data[,2:40]

# Benchmark in-memory performance

In [4]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- cor(data)
    end <- Sys.time()
    print(end - start)
#}

Time difference of 10.24339 secs


In [5]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- prcomp(data, retx=FALSE)
    end <- Sys.time()
    print(end - start)
#}

Time difference of 9.52385 secs


In [6]:
start <- Sys.time()
res <- naiveBayes.train(x, y)
end <- Sys.time()
print(end - start)

Time difference of 4.181429 secs


In [8]:
gc()
start <- Sys.time()
res <- logistic.regression(x, y, method="L-BFGS-B", max.iters=12)
end <- Sys.time()
print(end - start)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1424920,76.1,2164898,115.7,2164898,115.7
Vcells,1418510,10.9,2382142,18.2,1990277,15.2


Iteration 1: 
fx = 0.345227

  xnorm = 1, gnorm = 0.237696, step = 2.13487

Iteration 2: 
fx = 0.191088

  xnorm = 2.04532, gnorm = 0.106568, step = 1

Iteration 3: 
fx = 0.160283

  xnorm = 2.8301, gnorm = 0.0997518, step = 1

Iteration 4: 
fx = 0.14168

  xnorm = 3.16955, gnorm = 0.0269789, step = 1

Iteration 5: 
fx = 0.138995

  xnorm = 3.342, gnorm = 0.0110867, step = 1

Iteration 6: 
fx = 0.138094

  xnorm = 3.47367, gnorm = 0.00461146, step = 1

Iteration 7: 
fx = 0.1378

  xnorm = 3.53629, gnorm = 0.0035362, step = 1

Iteration 8: 
fx = 0.137676

  xnorm = 3.59411, gnorm = 0.00502946, step = 1

Iteration 9: 
fx = 0.13758

  xnorm = 3.59036, gnorm = 0.00184809, step = 1

Iteration 10: 
fx = 0.137556

  xnorm = 3.58265, gnorm = 0.00119976, step = 1

Iteration 11: 
fx = 0.137538

  xnorm = 3.57972, gnorm = 0.00119646, step = 1

Iteration 12: 
fx = 0.137516

  xnorm = 3.58036, gnorm = 0.00125034, step = 1

L-BFGS optimization terminated with status code = -997
fx = 0.137516



In [10]:
data1 <- fm.runif.matrix(336000000, 32, min = 0, max=1000)

In [11]:
start <- Sys.time()
res <- fm.kmeans(data1, 10, iter.max = 5, use.blas = TRUE)
end <- Sys.time()
print(end - start)

KMeans takes 5 iterations and 30.0517 seconds
Time difference of 40.61406 secs


In [19]:
esti.cov.full <- function(X, resp, nk, means, reg.covar)
{
	k <- nrow(means)
	d <- ncol(means)
	covs <- list()
	for (i in 1:k) {
		diff <- sweep(X, 2, means[i,], "-")
		covs[[i]] <- t(resp[,i] * diff) %*% diff / nk[i]
	}
	lapply(covs, function(x) as.matrix(x) + diag(rep(reg.covar, d)))
}

# In this case, we assume all Gaussian distribution has the same covariance matrix.
esti.cov.tied <- function(X, resp, nk, means, reg.covar)
{
	avg.X2 <- t(X) %*% X
	avg.means2 <- t(means * nk) %*% means
	as.matrix((avg.X2 - avg.means2) / sum(nk)) + diag(rep(reg.covar, d))
}

esti.cov.diag <- function(X, resp, nk, means, reg.covar)
{
	avg.X2 <- t(resp) %*% (X * X) / nk
	avg.means2 <- means^2
	avg.Xmeans <- means * (t(resp) %*% X) / nk
	covars <- as.matrix(avg.X2 - 2*avg.Xmeans + avg.means2)
	ifelse(covars < reg.covar, reg.covar, covars)
}

esti.cov.spherical <- function(X, resp, nk, means, reg.covar)
{
	rowMeans(esti.cov.diag(X, resp, nk, means, reg.covar))
}

# This estimate the parameters of Mixture of Gaussian
esti.gaussian.params <- function(X, resp, reg.covar, cov.type)
{
	n <- nrow(X)
	nk <- colSums(resp)
	# a k x d matrix. Each row is the mean of a component.
	means <- t(resp) %*% X / nk
	# a list of covariance matrices for all components.
	covs <- if (cov.type == "full") esti.cov.full(X, resp, nk, means, reg.covar)
		else if (cov.type == "tied") esti.cov.tied(X, resp, nk, means, reg.covar)
		else if (cov.type == "diag") esti.cov.diag(X, resp, nk, means, reg.covar)
		else if (cov.type == "spherical") esti.cov.spherical(X, resp, nk, means, reg.covar)
		else NULL
	list(weights=as.vector(nk)/n, means=as.matrix(means), covs=covs)
}

init.params <- function(X, k, reg.covar, method, cov.type)
{
	N <- nrow(X)
	if (method == "kmeans") {
		res <- fm.kmeans(X, k, 100)
		resp <- fm.matrix(0, nrow(X), k)
		idx <- cbind(fm.seq.int(1, nrow(X), 1), res$cluster)
		resp[idx] <- 1
		# Estimate weights, means and covariances
		params <- esti.gaussian.params(X, resp, reg.covar, cov.type)
		list(weights=params$weights, means=params$means, covs=params$covs)
	}
	else if (method == "random") {
		resp <- fm.runif.matrix(N, k, in.mem=fm.in.mem(X))
		# each row needs to sum up to 1.
		resp <- resp / rowSums(resp)
		# Estimate weights, means and covariances
		params <- esti.gaussian.params(X, resp, reg.covar, cov.type)
		list(weights=params$weights, means=params$means, covs=params$covs)
	}
	else if (method == "random_params") {
		m <- dim(X)[1]
		rand.k <- floor(runif(k, 1, m))
		mus <- X[rand.k,]
		init.covar <- cov(X)
		covars <- list()
		for (i in 1:k)
			covars[[i]] <- init.covar
		phi <- rep.int(1/m, k)
		list(weights=phi, means=mus, covs=covars)
	}
	else
		stop("unknown init method")
}

est.logprob <- function(X, means, covars, cov.type)
{
	n <- nrow(X)
	d <- ncol(X)
	k <- nrow(means)

	comp.logprob.vec <- function(X, mu, covar.vec) {
		X1 <- sweep(X, 2, mu, "-")
		X2 <- sweep(X1, 2, covar.vec, "/")
		rowSums(X2 * X1)
	}

	if (cov.type == "full") {
		logprob <- list()
		for (i in 1:k)
			logprob[[i]] <- fm.dmvnorm(X, means[i,], covars[[i]], log=TRUE)
		return(do.call(cbind, logprob))
	}
	else if (cov.type == "tied") {
		logprob <- list()
		for (i in 1:k)
			logprob[[i]] <- fm.dmvnorm(X, means[i,], covars, log=TRUE)
		return(do.call(cbind, logprob))
	}
	else if (cov.type == "diag") {
		logprob <- list()
		for (i in 1:k) {
			logprob[[i]] <- comp.logprob.vec(X, means[i,], covars[i,])
			logprob[[i]] <- -sum(log(covars[i,]))/2 - 0.5 * (d * log(2 * pi) + logprob[[i]])
		}
		return(do.call(cbind, logprob))
	}
	else if (cov.type == "spherical") {
		logprob <- list()
		for (i in 1:k) {
			logprob[[i]] <- comp.logprob.vec(X, means[i,], rep(covars[i], d))
			logprob[[i]] <- -log(covars[i])*d/2 - 0.5 * (d * log(2 * pi) + logprob[[i]])
		}
		return(do.call(cbind, logprob))
	}
}

est.weighted.logprob <- function(X, means, covars, cov.type, weights)
{
	ret <- est.logprob(X, means, covars, cov.type)
#	ret <- fm.materialize(est.logprob(X, means, covars, cov.type))
	sweep(ret, 2, log(weights), "+")
}

logsumexp <- function(X)
{
	max.X <- fm.agg.mat(X, 1, fm.bo.max)
	log(rowSums(exp(X - max.X))) + max.X
}

# Estimate the log likelihood
# @return norm
# @return resp is a n x k matrix. It indicates the probability
#        that a data point belongs to a cluster.
fm.estep <- function(X, params, cov.type)
{
	weighted.logprob <- est.weighted.logprob(X, params$means,
											 params$covs, cov.type,
											 params$weights)
	logprob.norm <- logsumexp(weighted.logprob)
	log.resp <- weighted.logprob - logprob.norm
	norm <- mean(logprob.norm)
	fm.materialize(log.resp, logprob.norm, norm)
	list(norm=as.vector(norm), resp=log.resp)
}

# This estimate the parameters.
fm.mstep <- function(X, log.resp, reg.covar, cov.type)
{
	params <- esti.gaussian.params(X, exp(log.resp), reg.covar, cov.type)
	list(weights=params$weights, means=params$means, covs=params$covs)
}

compute.lower.bound <- function(log.resp, log.norm)
{
	log.norm
}

# Fit a mixture of Gaussian distribution on the data.
#
# @param X is a n x d matrix. It's the input data.
# @param k the number of components.
# @param reg.covar is a real value. It's added to the diagonal of
#        the covariance matrix to make it non-singular.
# @param cov.type is the type of covariance matrix. It can be
#        one of {"full", "tied", "diag", "spherical"}.
#        \itemize{
#        \item{"full"}{each component has its own general covariance matrix.}
#        \item{"tied"}{all components share the same general covariance matrix.}
#        \item{"diag"}{each component has its own diagonal covariance matrix.}
#        \item{"spherical"}{each component has its own single variance.}
#        }
# @return 
#        \itemize{
#        \item{loglik}{a n x k matrix, whose \code{[i, k]}th entry is
#                the conditional probability of the ith observation
#                belonging to the kth component of the mixture.}
#        \item{iter}{the number of iterations}
#        \item{parameters}{parameters of the mixture of Gaussian distribution.
#             \itemize{
#             \item{weights}{a vector of k elements. Each element is
#              the weight of the Gaussian distribution in the mixture.}
#             \item{means}{a k x d matrix. Each row is the mean of a Gaussian distribution.}
#             \item{covs}{a list of matrices, a matrix or a vector, depending on \code{cov.type}}}
#        }
GMM.fit <- function(X, k, max.iter=100, tol=1e-3, reg.covar=1e-6,
					method=c("random", "random_params", "kmeans"),
					cov.type=c("full", "tied", "diag", "spherical"))
{
	method <- match.arg(method)
	cov.type <- match.arg(cov.type)
	params <- init.params(X, k, reg.covar, method, cov.type)
	for (i in 1:max.iter) {
		eret <- fm.estep(X, params, cov.type)
		params <- fm.mstep(X, eret$resp, reg.covar, cov.type)
		lb <- compute.lower.bound(eret$resp, eret$norm)
		if (i > 5 && abs(lb - prev.lb) < tol)
			break
		prev.lb <- lb
		gc()
	}
	structure(list(loglik=eret$resp, score=eret$norm, iter=i,
				   cov.type=cov.type, parameters=params), class="GMM")
}


In [20]:
start <- Sys.time()
res <- GMM.fit(data1, 10, max.iter = 5, method = "random_params")
end <- Sys.time()
print(end - start)

Time difference of 20.30929 mins


# Benchmark external-memory performance

In [12]:
y <- emdata[,1]
x <- emdata[,2:40]

In [13]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- cor(emdata)
    end <- Sys.time()
    print(end - start)
#}

Time difference of 10.60668 secs


In [14]:
#for (i in 1:5) {
    start <- Sys.time()
    res <- prcomp(data, retx=FALSE)
    end <- Sys.time()
    print(end - start)
#}

Time difference of 9.63978 secs


In [15]:
start <- Sys.time()
res <- naiveBayes.train(x, y)
end <- Sys.time()
print(end - start)

Time difference of 5.583336 secs


In [18]:
gc()
source("/FlashX/FlashR-learn/R/classification.R")
start <- Sys.time()
res <- logistic.regression(x, y, method="L-BFGS-B", max.iters=12)
end <- Sys.time()
print(end - start)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1438900,76.9,2164898,115.7,2164898,115.7
Vcells,1443915,11.1,2382142,18.2,1990277,15.2


Iteration 1: 
fx = 0.345227

  xnorm = 1, gnorm = 0.237696, step = 2.13487

Iteration 2: 
fx = 0.191088

  xnorm = 2.04532, gnorm = 0.106568, step = 1

Iteration 3: 
fx = 0.160283

  xnorm = 2.8301, gnorm = 0.0997518, step = 1

Iteration 4: 
fx = 0.14168

  xnorm = 3.16955, gnorm = 0.0269789, step = 1

Iteration 5: 
fx = 0.138995

  xnorm = 3.342, gnorm = 0.0110867, step = 1

Iteration 6: 
fx = 0.138094

  xnorm = 3.47367, gnorm = 0.00461146, step = 1

Iteration 7: 
fx = 0.1378

  xnorm = 3.53629, gnorm = 0.0035362, step = 1

Iteration 8: 
fx = 0.137676

  xnorm = 3.59411, gnorm = 0.00502946, step = 1

Iteration 9: 
fx = 0.13758

  xnorm = 3.59036, gnorm = 0.00184809, step = 1

Iteration 10: 
fx = 0.137556

  xnorm = 3.58265, gnorm = 0.00119976, step = 1

Iteration 11: 
fx = 0.137538

  xnorm = 3.57972, gnorm = 0.00119646, step = 1

Iteration 12: 
fx = 0.137516

  xnorm = 3.58036, gnorm = 0.00125034, step = 1

L-BFGS optimization terminated with status code = -997
fx = 0.137516

Time d

In [19]:
emdata1 <- fm.conv.store(data1, in.mem = FALSE)

In [20]:
start <- Sys.time()
res <- fm.kmeans(emdata1, 10, iter.max = 5, use.blas = TRUE)
end <- Sys.time()
print(end - start)

KMeans takes 5 iterations and 41.57616 seconds
Time difference of 1.248347 mins


In [21]:
start <- Sys.time()
res <- GMM.fit(emdata1, 10, max.iter = 5, method = "random_params")
end <- Sys.time()
print(end - start)

Time difference of 20.80116 mins


In [21]:
fm.start.profiler("/tmp/logistic.prof")